<a href="https://colab.research.google.com/github/snehanshastri/BIS/blob/main/CuckooSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
from scipy.special import gamma  # Import the gamma function

# Sphere Function to optimize
def sphere(x):
    return np.sum(x**2)

# Initialize population
def initialize_population(n, dim, bounds):
    population = np.random.uniform(bounds[0], bounds[1], (n, dim))
    return population

# Lévy flight for exploration
def levy_flight(Lambda=1.5):
    sigma = (gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) / gamma((1 + Lambda) / 2) * Lambda ** ((Lambda - 1) / 2)) ** (1 / Lambda)  # Use gamma instead of Gamma
    u = np.random.normal(0, sigma, size=1)
    v = np.random.normal(0, 1, size=1)
    step = u / (np.abs(v) ** (1 / Lambda))
    return step

# Cuckoo Search Algorithm
def cuckoo_search(func, n=50, dim=5, max_iter=1000, bounds=(-5.12, 5.12), pa=0.25):
    # Initialize nests (population)
    nests = initialize_population(n, dim, bounds)

    # Evaluate fitness of the population
    fitness = np.array([func(nest) for nest in nests])

    # Best solution
    best_idx = np.argmin(fitness)
    best_nest = nests[best_idx]
    best_fitness = fitness[best_idx]

    # Main loop
    for _ in range(max_iter):
        # Generate new solutions (cuckoo search)
        new_nests = np.copy(nests)

        for i in range(n):
            # Lévy flight for generating new solutions
            step = levy_flight()
            new_nests[i] = nests[i] + step * (nests[i] - best_nest)  # Move towards the best nest

            # Boundary control
            new_nests[i] = np.clip(new_nests[i], bounds[0], bounds[1])

            # Evaluate new fitness
            new_fitness = func(new_nests[i])

            # If new solution is better, replace it
            if new_fitness < fitness[i]:
                nests[i] = new_nests[i]
                fitness[i] = new_fitness

        # Abandon some nests (based on the probability pa)
        for i in range(n):
            if random.random() < pa:
                nests[i] = initialize_population(1, dim, bounds)[0]  # Reinitialize with new random solution
                fitness[i] = func(nests[i])

        # Update the best solution
        best_idx = np.argmin(fitness)
        best_nest = nests[best_idx]
        best_fitness = fitness[best_idx]

    return best_nest, best_fitness

# Main execution
if __name__ == "__main__":
    best_solution, best_value = cuckoo_search(sphere, n=50, dim=10, max_iter=1000, bounds=(-5.12, 5.12), pa=0.25)
    print("Best Solution: ", best_solution)
    print("Best Fitness (Value of Sphere Function): ", best_value)


Best Solution:  [-0.02122667  0.03639467 -0.06056118 -0.02738857 -0.00015126  0.04255517
 -0.06440598 -0.01718141  0.00703945  0.00599116]
Best Fitness (Value of Sphere Function):  0.012532678828215921
